In [1]:
!pip install tensorflow keras opencv-python transformers pandas numpy nltk

  Using cached numpy-1.26.4-cp312-cp312-win_amd64.whl.metadata (61 kB)
  Using cached wheel-0.44.0-py3-none-any.whl.metadata (2.3 kB)
   ---------------------------------------- 0.0/385.2 MB ? eta -:--:--
   ---------------------------------------- 4.2/385.2 MB 22.9 MB/s eta 0:00:17
   - -------------------------------------- 11.8/385.2 MB 29.5 MB/s eta 0:00:13
   - -------------------------------------- 18.9/385.2 MB 30.5 MB/s eta 0:00:13
   -- ------------------------------------- 28.6/385.2 MB 34.2 MB/s eta 0:00:11
   --- ------------------------------------ 30.1/385.2 MB 29.0 MB/s eta 0:00:13
   ---- ----------------------------------- 39.1/385.2 MB 30.6 MB/s eta 0:00:12
   ---- ----------------------------------- 48.0/385.2 MB 32.5 MB/s eta 0:00:11
   ----- ---------------------------------- 57.7/385.2 MB 34.0 MB/s eta 0:00:10
   ------ --------------------------------- 67.4/385.2 MB 35.2 MB/s eta 0:00:10
   ------- -------------------------------- 76.5/385.2 MB 35.9 MB/s eta 0:00

In [2]:
# Import necessary libraries
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import img_to_array

In [3]:
# Path to your dataset (adjust the path if needed)
dataset_path = '../datasets/chest_xray/'

# Image dimensions and categories
IMG_SIZE = 224
categories = ['NORMAL', 'PNEUMONIA']

# Function to load and preprocess images
def load_images_from_folder(folder):
    data = []
    labels = []
    
    for category in categories:
        category_path = os.path.join(folder, category)
        class_num = categories.index(category)  # 0 for NORMAL, 1 for PNEUMONIA
        
        for img in os.listdir(category_path):
            try:
                img_array = cv2.imread(os.path.join(category_path, img), cv2.IMREAD_GRAYSCALE)
                img_resized = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
                data.append(img_to_array(img_resized))
                labels.append(class_num)
            except Exception as e:
                print(f"Error loading image {img}: {e}")
    
    return np.array(data), np.array(labels)

# Load train images
train_images, train_labels = load_images_from_folder(os.path.join(dataset_path, 'train'))

# Normalize the images (pixel values between 0 and 1)
train_images = train_images / 255.0

# Convert labels to one-hot encoding
train_labels = to_categorical(train_labels, num_classes=2)

In [4]:
# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(train_images, train_labels, test_size=0.2, random_state=42)

In [5]:
# Build the CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 1)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(2, activation='softmax')  # Output layer for two classes (NORMAL, PNEUMONIA)
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Show model summary
model.summary()

C:\Users\ADITYA SINGH\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 222, 222, 32)        │             320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 111, 111, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 109, 109, 64)        │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 54, 54, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 186624)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │      23,888,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 2)                   │             258 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 23,907,074 (91.20 MB)

 Trainable params: 23,907,074 (91.20 MB)

 Non-trainable params: 0 (0.00 B)

In [6]:
# Train the model
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10, batch_size=32)

# Save the model
model.save('../models/cnn_pneumonia_model.h5')


Epoch 1/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 102s 740ms/step - accuracy: 0.8320 - loss: 0.7316 - val_accuracy: 0.9521 - val_loss: 0.1108
Epoch 2/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 93s 708ms/step - accuracy: 0.9605 - loss: 0.1186 - val_accuracy: 0.9540 - val_loss: 0.1167
Epoch 3/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 92s 702ms/step - accuracy: 0.9727 - loss: 0.0788 - val_accuracy: 0.9607 - val_loss: 0.1101
Epoch 4/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 99s 756ms/step - accuracy: 0.9740 - loss: 0.0721 - val_accuracy: 0.9559 - val_loss: 0.1219
Epoch 5/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 100s 762ms/step - accuracy: 0.9810 - loss: 0.0570 - val_accuracy: 0.9607 - val_loss: 0.1093
Epoch 6/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 90s 686ms/step - accuracy: 0.9910 - loss: 0.0284 - val_accuracy: 0.9655 - val_loss: 0.1167
Epoch 7/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 87s 666ms/step - accuracy: 0.9891 - loss: 0.0278 - val_accuracy: 0.9703 - val_loss: 0.1177
Epoch 8/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 84s 638ms/step - accuracy: 0.9907 - loss:

In [7]:
# Evaluate the model
test_images, test_labels = load_images_from_folder(os.path.join(dataset_path, 'test'))
test_images = test_images / 255.0
test_labels = to_categorical(test_labels, num_classes=2)

test_loss, test_acc = model.evaluate(test_images, test_labels)
print(f"Test accuracy: {test_acc:.2f}") 

20/20 ━━━━━━━━━━━━━━━━━━━━ 3s 160ms/step - accuracy: 0.4635 - loss: 5.0390
Test accuracy: 0.71
